In [6]:
import numpy as np
import cv2

In [7]:
face_detection_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt','./models/res10_300x300_ssd_iter_140000_fp16.caffemodel')


In [8]:
def face_detection_dnn(img):

    blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123), swapRB=True, crop=False)
    #(104, 177, 123) => These are the Mean Values for Preprocessing.
    #They’re used to normalize the image before passing it to the neural network.
    face_detection_model.setInput(blob)  # Feed the preprocessed image in
    detections = face_detection_model.forward()  # Get predictions (like face boxes)
    #7 values	[id, class, confidence, x1, y1, x2, y2] => confidence score is something between 0 and 1
    image = img.copy()
    
    h,w = image.shape[0:2]
    
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
             box = detections[0, 0, i, 3:7]*[w, h, w, h] 
             box = box.astype("int")
             pt1 = (box[0], box[1])
             pt2 =(box[2],box[3])
             cv2.rectangle(image,pt1,pt2,(0,255,0),2)
             text = f'score : {round(confidence * 100)} %'
             cv2.putText(image,text,pt1,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    
    return image


In [9]:
cap = cv2.VideoCapture(0)
while True:
    ret , frame = cap.read()
    
    if ret == False:
        break
    cv2.imshow('Live DNN', face_detection_dnn(frame))
    if cv2.waitKey(1) == ord ('q'):
        break

cap.release()
cv2.destroyAllWindows()

![Sample Output](sample.png)